# Explainable AI Final Project LIME Implmentation
### Katie Hucker (kh509)

In this notebook we have the implmentation code to run LIME interpretations for Nuscenes traffic cone images. The LIME interpretations are ran on a finetuned YOLOv8 model that was trained to detect the traffic cones within the dataset.

The first section defines the code: functions, process, the HOW we get the LIME interpretations.

Then we compare 2 visibility level traffic cones: occluded (1-40% visible) and visible (80-100%). This is important to my Capstone groups analysis of how YOLO performs for very occluded traffic cones.  

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1lZsIpz-v9Sx-ZUYcY8clLkIFKvkO5fwu?usp=sharing)

# Section 1: The Code - How can we implement LIME on our dataset?

## Installs Imports and Mounting

In [ ]:
#installs uncomment if you do NOT need
!pip install ultralytics lime torch

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
from ultralytics import YOLO
import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries
import cv2
from skimage.segmentation import mark_boundaries, felzenszwalb, slic, quickshift

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Imports and Configs
model_path = "/content/drive/MyDrive/Capstone/explainable models/best.pt"  # Path to your model
image_path = "/content/drive/MyDrive/Capstone/explainable models/images/n015-2018-07-18-11-41-49+0800__CAM_BACK__1531885800937525.jpg"  # Path to image

In [ ]:
# Load model
print(f"Loading YOLOv8 model from: {model_path}")
model = YOLO(model_path)
model.to(device)
print(f"Model loaded with {len(model.names)} classes: {model.names}")

## Set-Up Functions

This function runs the YOLO model on the input Nuscenes image. It extracts the detected class IDs and their confidence scores, sorts them in descending order by confidence, and returns the detections along with the raw results and original image.

In [ ]:
def cone_detect(model, image,  conf_threshold=0.01):
    img_np = image
    results = model(img_np, conf=conf_threshold)

    # Extract detected classes
    result = results[0]
    detected_classes = []

    for box in result.boxes:
        class_id = int(box.cls[0].item())
        confidence = box.conf[0].item()
        detected_classes.append((class_id, confidence))

    # Sort by confidence
    detected_classes.sort(key=lambda x: x[1], reverse=True)


    return detected_classes, results, img_np

This function segments the input image into superpixels using a selected method such as SLIC, Felzenszwalb, or Quickshift. These segments are used by LIME to perturb and explain specific regions of the image relevant to the model's predictions.

In [ ]:
def create_segmentation(image, method='slic', n_segments=50, compactness=15, sigma=1.5):
  # https://scikit-image.org/docs/stable/api/skimage.segmentation.html
    if method == 'slic':
        segments = slic(image, n_segments=n_segments, compactness=compactness,
                        sigma=sigma, start_label=1)
    elif method == 'felzenszwalb':
        segments = felzenszwalb(image, scale=100, sigma=sigma, min_size=50)
    elif method == 'quickshift':
        segments = quickshift(image, kernel_size=3, max_dist=6, ratio=0.5)

    return segments

## LIME Definitions

These functions initialize and return a LIME image explainer object and the results. The LIME image explainer has a specified kernel width, which controls the locality of the explanation. A smaller kernel width makes the explanation more sensitive to closer perturbations of the input image. Then we use this object to generates a LIME explanation for the given image by applying random perturbations over the provided segments and using the model's prediction function.




In [ ]:
def create_lime_explainer(kernel_width=0.5):

    return lime_image.LimeImageExplainer(kernel_width=kernel_width)

def get_lime_explanation(explainer, image, predict_function, segments, top_labels=1, num_samples=500, batch_size=16):
    """
    Get LIME explanation for an image.

    """
    explanation = explainer.explain_instance(
        image,
        predict_function,
        top_labels=top_labels,
        hide_color=0,
        num_samples=num_samples,
        batch_size=batch_size,
        segmentation_fn=lambda x: segments
    )
    return explanation

## LIME Implementation

This function is used to generate predictions for LIME by combining what the YOLO model detects and where those detections happen in the image. It uses the segments to guide the LIME explanations It follows these steps:
  1. For each image, it runs YOLO to get the detected objects and their confidence scores.
  2. It breaks the image into a grid and builds a heatmap showing which parts of the image each class was detected in. This helps keep track of both the confidence of a detection and its location.
  3. The result is a prediction vector for each image that reflects how confident the model is about each class, taking into account both what it saw and where it saw it. This makes LIME explanations more meaningful by adding spatial context.

In [ ]:
import numpy as np
import cv2
from lime import lime_image
import skimage

In [ ]:
def predict_fn(images, model, class_names):
  # This function was edited from a 2025 MIDS Capstone project generated by me to display confidences and detected box.
  # It previously worked with multiple object classes so some of that functionality is still there.

    batch_preds = []

    for img in images:
        #int type check
        img_for_model = img.astype(np.uint8)

        # predict
        results = model(img_for_model)
        result = results[0]

        # Get the image dimensions
        img_height, img_width = img.shape[:2]

        #inits
        grid_size = 20
        confidence_array = np.zeros(len(class_names))
        heatmap = np.zeros((grid_size, grid_size, len(class_names)))

        # For each box in image
        for box in result.boxes:
            class_id = int(box.cls[0].item())
            confidence = box.conf[0].item()

            # get the coordinates of the detected box
            x1, y1, x2, y2 = box.xyxy[0].tolist()

            # Claude Sonnet 3.7 generated this code on 2/13/2025
            grid_x1 = int((x1 / img_width) * grid_size)
            grid_y1 = int((y1 / img_height) * grid_size)
            grid_x2 = int((x2 / img_width) * grid_size)
            grid_y2 = int((y2 / img_height) * grid_size)

            # Claude Sonnet 3.7 generated this code on 2/13/2025
            grid_x1 = max(0, min(grid_x1, grid_size-1))
            grid_y1 = max(0, min(grid_y1, grid_size-1))
            grid_x2 = max(0, min(grid_x2, grid_size-1))
            grid_y2 = max(0, min(grid_y2, grid_size-1))

            # Claude Sonnet 3.7 generated this code on 2/13/2025
            for gx in range(grid_x1, grid_x2+1):
                for gy in range(grid_y1, grid_y2+1):
                    heatmap[gy, gx, class_id] = max(heatmap[gy, gx, class_id], confidence)

            # Update the overall confidence
            confidence_array[class_id] = max(confidence_array[class_id], confidence)


        for class_id in range(len(class_names)):
            if confidence_array[class_id] > 0:
                # The presence of the class contributes to the prediction
                confidence_array[class_id] = confidence_array[class_id] * np.mean(heatmap[:,:,class_id] > 0)  # Claude Sonnet 3.7 was used to generate this line of code on 2/13/2025

        batch_preds.append(confidence_array)

    return np.array(batch_preds)


This function ties everything together to generate the LIME explanation for an our image and using the provided YOLO detection model.

  1. It calls the prediction wrapper first we defined this above.
  2. It then segments the image using a method like SLIC to create superpixels, which are important for LIME to know what parts of the image to modify.
  3. The LIME explainer runs to get an explanation of which parts of the image most influenced the model’s prediction.
  4. The function returns the explanation, the segmentation used, and the singular detected class.

In [ ]:
def analyze_image(image, model, class_names, segmentation_method='slic',
                 n_segments=50, num_samples=1000, batch_size=16):

    # defined above
    def predict_wrapper(images):
        return predict_fn(images, model, class_names)

    # defined above
    segments = create_segmentation(image, method=segmentation_method, n_segments=n_segments)

    # defined above
    explainer = create_lime_explainer()

    # defined above
    explanation = get_lime_explanation( # https://lime-ml.readthedocs.io/en/latest/lime.html#module-lime.explanation
        explainer, image, predict_wrapper, segments,
        num_samples=num_samples, batch_size=batch_size
    )

    if len(explanation.top_labels) > 0:
        top_class_id = explanation.top_labels[0]


    return explanation, segments, top_class_id


## Plotting and Output Code


In [ ]:

def plot_lime_basic(image, explanation, top_class_id, class_names, figsize=(15, 5)):

    if isinstance(class_names, (list, tuple, np.ndarray)) and len(class_names) > top_class_id:
        class_name = class_names[top_class_id]
    else:
        class_name = f"Class {top_class_id}"  # Claude Sonnet 3.7 was used to generate this line of Code on 4/15/2025


    temp, mask = explanation.get_image_and_mask(  #https://lime-ml.readthedocs.io/en/latest/lime.html#module-lime.explanation
        top_class_id,
        positive_only=False,
        num_features=10,
        hide_rest=False
    )

    plt.figure(figsize=figsize)

    #Regular image
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis('off')

    # Lime image with the segment boundaries
    plt.subplot(1, 2, 2)
    plt.imshow(mark_boundaries(temp / 255.0, mask))
    plt.title(f"LIME Explanation\nClass: {class_name}")
    plt.axis('off')

    plt.tight_layout()
    plt.show()



def enhanced_visualize_segments(image, explanation, label, threshold=0.01):
    ## Claude Sonnet 3.7 was used to generate this function to show the segments with numbers and the boundaries on the segments for more detailed analuysis
    ## Claude was asked "Please add numbers to the segments and provide borders to all the images and then provided the current working LIME code including the plot code above."
    ## This was completed on 4/17/2025 at 4:23pm.

    # Check if the label exists in the explanation
    if label not in explanation.local_exp:
        # If no explanation for this label, return original image
        return np.copy(image).astype(np.float32) / 255

    dict_heatmap = dict(explanation.local_exp[label])
    segments = explanation.segments

    # Normalize values for visualization
    values = np.array(list(dict_heatmap.values()))
    if len(values) == 0:
        return np.copy(image).astype(np.float32) / 255

    abs_values = np.abs(values)
    max_abs_val = abs_values.max() if abs_values.max() > 0 else 1.0  # Avoid division by zero

    # Create a copy of the image
    segment_image = np.copy(image).astype(np.float32) / 255

    # Create an overlay for segments
    overlay = np.zeros_like(segment_image)

    # Create a mask for all significant segments
    significant_mask = np.zeros(segments.shape, dtype=bool)

    # Color each segment based on importance
    for segment_id, value in dict_heatmap.items():
        # Skip segments with minimal contribution
        if abs(value) / max_abs_val < threshold:
            continue

        # Normalize value between -1 and 1
        normalized_value = value / max_abs_val

        # Choose color based on contribution (positive=green, negative=red)
        if normalized_value > 0:
            # Increase the intensity for better visibility
            color = [0, min(1.0, normalized_value * 2), 0]  # Green for positive
        else:
            # Increase the intensity for better visibility
            color = [min(1.0, -normalized_value * 2), 0, 0]  # Red for negative

        # Set color for the segment
        segment_mask = segments == segment_id
        overlay[segment_mask] = color
        significant_mask |= segment_mask

    # Blend original image with overlay, with increased contrast
    alpha = 0.7  # Higher transparency factor

    # Slightly dim parts of the image not in significant segments
    segment_image[~significant_mask] *= 0.7

    # Apply the colored overlay only on significant segments
    blended = segment_image.copy()
    if np.any(significant_mask):  # Only if we have significant segments
        blended[significant_mask] = (1 - alpha) * segment_image[significant_mask] + alpha * overlay[significant_mask]

    return np.clip(blended, 0, 1)


def plot_enhanced_visualization(image, segments, explanation, top_class_id, class_names, figsize=(18, 6), show_segment_numbers=True):
    ## Using the same prompt and time this function was generated with Claude Sonnet see above citation.


    if isinstance(class_names, (list, tuple, np.ndarray)) and len(class_names) > top_class_id:
        class_name = class_names[top_class_id]
    else:
        class_name = f"Class {top_class_id}"

    #calls above function
    enhanced_viz = enhanced_visualize_segments(image, explanation, top_class_id, threshold=0.01)

    plt.figure(figsize=figsize)


    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis('off')


    plt.subplot(1, 3, 2)
    plt.imshow(mark_boundaries(image/255.0, segments))
    plt.title(f"Segmentation")
    plt.axis('off')

    # Add segment numbers
    if show_segment_numbers:

        unique_segments = np.unique(segments)

        from scipy import ndimage

        for segment_id in unique_segments:

            mask = segments == segment_id

            if np.sum(mask) < 50:
                continue

            cy, cx = ndimage.center_of_mass(mask)

            plt.text(cx, cy, str(segment_id),
                     color='white', fontsize=8, ha='center', va='center',
                     bbox=dict(facecolor='black', alpha=0.5, pad=0))


    plt.subplot(1, 3, 3)
    plt.imshow(enhanced_viz)
    plt.title(f"Enhanced LIME Visualization for {class_name}\nGreen = Positive, Red = Negative")
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    # I added this code
    if top_class_id in explanation.local_exp:

        dict_heatmap = dict(explanation.local_exp[top_class_id])
        segment_importances = sorted(dict_heatmap.items(), key=lambda x: abs(x[1]), reverse=True)

        # Which segments have largest response sort then print
        print("\nMost important segments:")
        for segment_id, importance in segment_importances[:5]:
            direction = "Positive" if importance > 0 else "Negative"
            print(f"Segment {segment_id}: {direction} contribution of {abs(importance):.4f}")

    return


def print_explanation_summary(explanation, top_class_id, class_names):
    """
    Print numerical data like above but more detailed
    """
    if isinstance(class_names, (list, tuple, np.ndarray)) and len(class_names) > top_class_id:
        class_name = class_names[top_class_id]
    else:
        class_name = f"Class {top_class_id}"
    dict_heatmap = dict(explanation.local_exp[top_class_id])
    segment_importances = sorted(dict_heatmap.items(), key=lambda x: abs(x[1]), reverse=True)

    # Get the positive and negative contributions
    positive_segs = [s for s in segment_importances if s[1] > 0][:5]
    negative_segs = [s for s in segment_importances if s[1] < 0][:5]

    if positive_segs:
        print("\nTop Positive Contributing Segments:")
        for segment, importance in positive_segs:
            print(f"- Segment {segment}: contribution of {importance:.4f}")

    if negative_segs:
        print("\nTop Negative Contributing Segments:")
        for segment, importance in negative_segs:
            print(f"- Segment {segment}: contribution of {importance:.4f}")

    # Claude 3.7 generated this math code on 4/17/2025 on 4:56pm
    if positive_segs or negative_segs:
        total_positive = sum(imp for _, imp in positive_segs)
        total_negative = sum(imp for _, imp in negative_segs)
        total_abs = abs(total_positive) + abs(total_negative)
        pos_percentage = (abs(total_positive) / total_abs) * 100 if total_abs > 0 else 0
        neg_percentage = (abs(total_negative) / total_abs) * 100 if total_abs > 0 else 0

        print(f"\nPositive contributions account for approximately {pos_percentage:.1f}% of total influence")
        print(f"Negative contributions account for approximately {neg_percentage:.1f}% of total influence")

## Call functions

In [ ]:
#set-up image
img = Image.open(image_path).convert("RGB")
img_np = np.array(img)

In [ ]:
detected_classes, results, img_np = cone_detect(model, img_np)

In [ ]:
segments = create_segmentation(img_np)

In [ ]:
explanation, segments, class_names = analyze_image(img_np, model, ['traffic_cone'])

In [ ]:
top_class_id = detected_classes[0][0]
plot_lime_basic(img_np, explanation, top_class_id, class_names)

In [ ]:

plot_enhanced_visualization(img_np, segments, explanation, top_class_id, class_names)

In [ ]:
print_explanation_summary(explanation, top_class_id, class_names)

# Section 2: Occluded Cone Analysis

## Visible Cone

In [ ]:
# Imports and Configs
model_path = "/content/drive/MyDrive/Capstone/explainable models/best.pt"  # Path to your model
image_path = "/content/drive/MyDrive/Capstone/explainable models/images/n008-2018-08-01-15-16-36-0400__CAM_BACK_RIGHT__1533151427028113.jpg" # Path to image

In [ ]:
#set-up image
img = Image.open(image_path).convert("RGB")
img_np = np.array(img)

In [ ]:
detected_classes, results, img_np = cone_detect(model, img_np)

In [ ]:
segments = create_segmentation(img_np)

In [ ]:
explanation, segments, class_names = analyze_image(img_np, model, ['traffic_cone'])

In [ ]:
top_class_id = detected_classes[0][0]
plot_lime_basic(img_np, explanation, top_class_id, class_names)

In [ ]:

plot_enhanced_visualization(img_np, segments, explanation, top_class_id, class_names)

In [ ]:
print_explanation_summary(explanation, top_class_id, class_names)

## Discussion

We ran the LIME samples for 1000 iterations with 50 segments at batch size 16.  I wanted to keep the methods somewhat equal in this sense, however,LIME crashed at 5000 samples with A100 GPU so I went down to 1000. We see above that the positive contributions in green are more focused on the cones and the red shows up on that in between part which is a good sign that YOLO is recognizing thats when the cone ends and the concrete begins. The contribution numerics are very small to the point where we ask ourselves why that is so sensitive but its still able to detect the visible cones. LIME sees more positive contribution than other methods. More specifically we see the the van colored in positive contribution for YOLO. I wonder if this is due to the color its recognizing this NOT a traffic cone so its helping the deteciton. The numbered segments give is more detail on specific regions which help or don't help.

## Occluded Cone

In [ ]:
image_path="/content/drive/MyDrive/Capstone/explainable models/images/n015-2018-07-11-11-54-16+0800__CAM_BACK_LEFT__1531281493197423.jpg"
img = Image.open(image_path).convert("RGB")
img_np = np.array(img)

In [ ]:
detected_classes, results, img_np = cone_detect(model, img_np)

In [ ]:
segments = create_segmentation(img_np)

In [ ]:
class_names = ['traffic_cone']
def predict_wrapper(images):
    return predict_fn(images, model, class_names )

# Get LIME explanation
explainer = create_lime_explainer()
explanation = get_lime_explanation(explainer, img_np, predict_wrapper, segments)

In [ ]:
top_class_id = detected_classes[0][0]
plot_lime_basic(img_np, explanation, top_class_id, class_names)

In [ ]:

plot_enhanced_visualization(img_np, segments, explanation, top_class_id, class_names)

In [ ]:
print_explanation_summary(explanation, top_class_id, class_names)

## Occluded Cone Discussion

LIME shows a lot of positive contributions in the occluded cone images, however, the strongest are where the cones are. I feel like this image as more response in general than the other 2 and all we really gain is that LIME did not like the drain pipe region, but otherwise it found the cones with positive response as well as other parts of the image.

## Overall Discussion

LIME took a long time to run at 1000 samples, it was ran with segmentations to try to find regions of interest and though this is beneficial when we see the red segment in the visible cones, we see a lot of positive segments in both occluded and visible. There is really no difference between the cone visibility and this method. In the the occlded cones even thougbh section 35 does not have a cone at all LIME is showing a strong response there with YOLO like the neighboring sections.



## References
Claude Sonnet 3.7 was used when needed especially with plotting. It is cited throughout.


https://lime-ml.readthedocs.io/en/latest/lime.html

https://github.com/ultralytics/yolov5/issues/1991

https://medium.com/@shreeraj260405/hands-on-lime-practical-implementation-for-image-text-and-tabular-data-95566da87f57

